In [65]:
import pandas as pd
import numpy as np
import re
import sklearn.metrics as metrics
import pymorphy2
import nltk
from bs4 import BeautifulSoup

In [45]:
from scipy.spatial.distance import cdist
from scipy import sparse
from functools import reduce
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, cross_validate, KFold, cross_val_score, GroupKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import SGDClassifier
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.summarization.bm25 import get_bm25_weights, iter_bm25_bow
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest 
from sklearn.cluster import AgglomerativeClustering

In [3]:
def get_df():
    
    titles_df = pd.read_csv('./data/docs_titles.tsv/docs_titles.tsv', sep='\t')
    docs_id_test = pd.read_csv('./data/test_groups.csv', sep=',')
    docs_id_train = pd.read_csv('./data/train_groups.csv', sep=',')

    info = pd.concat([docs_id_train, docs_id_test])
    info.reset_index(drop=True)

    titles = pd.merge(titles_df, info[['group_id', 'doc_id', 'target']], on='doc_id', how='inner')
    titles['title'] += ' '
    titles['title'].fillna(' ', inplace=True)
    
    return titles

In [4]:
def titles_extraction():
    
    titles = get_df();

    titles = titles[['title', 'group_id']].groupby('group_id').sum()['title']

    titles = titles.apply(lambda x: cleaner(x))
    titles = titles.values

    return titles

In [5]:
def cleaning(titles, group_num, bad_words):
    titles = titles.lower()
    titles = re.sub(r'\W', '  ', titles)
    
    for i in bad_words[group_num]:
            titles = titles.replace(i, ' ')
    titles = re.sub(r'\s+', ' ', titles)
    return titles

In [6]:
# стеммниг для русского языка (взял в интернете)
class Porter:
    PERFECTIVEGROUND =  re.compile(u"((ив|ивши|ившись|ыв|ывши|ывшись)|((?<=[ая])(в|вши|вшись)))$")
    REFLEXIVE = re.compile(u"(с[яь])$")
    ADJECTIVE = re.compile(u"(ее|ие|ые|ое|ими|ыми|ей|ий|ый|ой|ем|им|ым|ом|его|ого|ему|ому|их|ых|ую|юю|ая|яя|ою|ею)$")
    PARTICIPLE = re.compile(u"((ивш|ывш|ующ)|((?<=[ая])(ем|нн|вш|ющ|щ)))$")
    VERB = re.compile(u"((ила|ыла|ена|ейте|уйте|ите|или|ыли|ей|уй|ил|ыл|им|ым|ен|ило|ыло|ено|ят|ует|уют|ит|ыт|ены|ить|ыть|ишь|ую|ю)|((?<=[ая])(ла|на|ете|йте|ли|й|л|ем|н|ло|но|ет|ют|ны|ть|ешь|нно)))$")
    NOUN = re.compile(u"(а|ев|ов|ие|ье|е|иями|ями|ами|еи|ии|и|ией|ей|ой|ий|й|иям|ям|ием|ем|ам|ом|о|у|ах|иях|ях|ы|ь|ию|ью|ю|ия|ья|я)$")
    RVRE = re.compile(u"^(.*?[аеиоуыэюя])(.*)$")
    DERIVATIONAL = re.compile(u".*[^аеиоуыэюя]+[аеиоуыэюя].*ость?$")
    DER = re.compile(u"ость?$")
    SUPERLATIVE = re.compile(u"(ейше|ейш)$")
    I = re.compile(u"и$")
    P = re.compile(u"ь$")
    NN = re.compile(u"нн$")

    def stem(string):
        
        changed = ''
        
        string = string.lower()

        for word in string.split():
            if not word.isdigit():
                word = word.replace(u'ё', u'е')
                m = re.match(Porter.RVRE, word)

                if m and m.groups():
                    pre = m.group(1)
                    rv = m.group(2)
                    temp = Porter.PERFECTIVEGROUND.sub('', rv, 1)
                    if temp == rv:
                        rv = Porter.REFLEXIVE.sub('', rv, 1)
                        temp = Porter.ADJECTIVE.sub('', rv, 1)
                        if temp != rv:
                            rv = temp
                            rv = Porter.PARTICIPLE.sub('', rv, 1)
                        else:
                            temp = Porter.VERB.sub('', rv, 1)
                            if temp == rv:
                                rv = Porter.NOUN.sub('', rv, 1)
                            else:
                                rv = temp
                    else:
                        rv = temp

                    rv = Porter.I.sub('', rv, 1)

                    if re.match(Porter.DERIVATIONAL, rv):
                        rv = Porter.DER.sub('', rv, 1)

                    temp = Porter.P.sub('', rv, 1)
                    if temp == rv:
                        rv = Porter.SUPERLATIVE.sub('', rv, 1)
                        rv = Porter.NN.sub(u'н', rv, 1)
                    else:
                        rv = temp
                    word = pre+rv

            changed += word + ' '

        return changed

In [7]:
# эта и 3 следующие функции используются для подсчета расстояний без тф идф и косинусной метрики

def features_create(mode, groups_titledata):
    
    if mode == 'train':
        y = []      
    X = []
    groups_train = []
    
    for new_group in groups_titledata:
        docs = groups_titledata[new_group]
        
        for k, info in enumerate(docs):
            
            doc_id = info[0]
            title = info[1]
            
            if mode == 'train':
                target_id = info[2]
                y.append(target_id)
                
            groups_train.append(new_group)
            all_dist = []
            words = set(title.strip().split())
            
            for j in range(0, len(docs)):
                if k == j:
                    continue
                info = docs[j]
                doc_id_j = info[0]
                title_j = info[1]

                words_j = set(title_j.strip().split())
                all_dist.append(len(words.intersection(words_j)))
                
            X.append(sorted(all_dist, reverse=True)[0:25])
            
    X = np.array(X)
    
    if mode == 'train':
        y = np.array(y)
    
    groups_train = np.array(groups_train)

    if mode == 'train':
        print(X.shape, y.shape, groups_train.shape)
        return X, y, groups_train
    else:
        print(X.shape, groups_train.shape)
        return X, groups_train

In [8]:
def tuple_x(a1, a2):
    return a1, a2

In [9]:
def title_info_dict(mode, doc_to_title):
    
    data = pd.read_csv('./data/{}_groups.csv'.format(mode))

    titledata = {}
    
    for i in range(len(data)):
        
        new_doc = data.iloc[i]
        doc_group = new_doc['group_id']
        doc_id = new_doc['doc_id']
            
        title = doc_to_title[doc_id]
        
        if doc_group not in titledata:
            titledata[doc_group] = []
            
        if mode == 'train':
            titledata[doc_group].append((doc_id, title, new_doc['target']))
        else:
            titledata[doc_group].append((doc_id, title))
        
    return titledata

In [10]:
def easy_launch():
    
    doc_to_title = {}
    with open('./data/unversal_table.csv', encoding = 'utf-8') as f:
        for num_line, line in enumerate(f):
            if num_line == 0:
                continue

            line = line.replace('\t', ',')
            data = line.strip().split(',')

            doc_id = int(data[0])
            if len(data) == 1:
                title = ''
            else:
                title = data[1]
            doc_to_title[doc_id] = title
            
    print('doc titles dict len = {}'.format(len(doc_to_title)))
    
    train_titledata = title_info_dict('train', doc_to_title)
    test_titledata = title_info_dict('test', doc_to_title)
    
    X_train, y_train, groups_train = features_create('train', train_titledata)
    X_test, groups_test = features_create('test', test_titledata)
    
    return X_train, y_train, X_test, groups_train, groups_test

In [11]:
# 3  функции для перебора комбинаций параметров
def flatten(x):
    
    result = []
    for elem in x:
        if hasattr(elem, "__iter__") and not isinstance(elem, str):
            result.extend(flatten(elem))
        else:
            result.append(elem)
            
    return result

In [12]:
def list_concat(list1, list2):
    
    len1 = len(list1)
    len2 = len(list2)
    
    return [[list1[i], list2[j]] for i in range(len1) for j in range(len2)]

In [13]:
def combinations(params):
    
    list_ = []
    for value in params.values():
        list_.append(value)

    tmp = reduce(lambda x, y: list_concat(x, y), list_)

    res = []
    for elem in tmp:
        param_list = flatten(elem)
        param_dict = dict(zip(params.keys(), param_list))
        res.append(param_dict)
        
    return res

In [14]:
def frange(start, stop, step):
    i = start
    while i < stop:
        yield i
        i += step

In [15]:
def validation(X_train, train_target, model, params, folds_gen_func, folds_num=5, thresholds=[0.32], **kwargs):
    
    scaler = StandardScaler()
    scaler.fit(X_train) 
    
    main_res = []
    for param_set in combinations(params):
        
        print(param_set)        
        exact_model = model(**param_set) 
        
        fold_generator = folds_gen_func(folds_num)
        
        for th in thresholds:
#           th = 0.27
            print('th = ', th)
            res = []
            for train_index, test_index in fold_generator.split(X_train, train_target, **kwargs):

                exact_model.fit(scaler.transform(X_train[train_index]), train_target[train_index])

                y_pred = [0 if val < th else 1 for val in exact_model.predict_proba(scaler.transform(X_train[test_index]))[:,1]]

                score = metrics.f1_score(train_target[test_index],\
                                                y_pred)
    #               print('threshold = {}, score = {}'.format(th, score))

                res.append(score)
        #                                       exact_model.predict(scaler.transform(X_train[test_index]))))


            mean = sum(res)/len(res)
            print(mean)
#             print('threshold = {}, score = {}'.format(th, mean))
            main_res.append((mean, param_set, th))
    
    best = main_res[np.argmax([res[0] for res in main_res])]
    print('--------max-------')
    print(best)
    
    return best

In [16]:
# Сохраняет решение

def save_submission(y_pred):

    data = pd.read_csv('data/test_groups.csv')
    print('len data = ', len(data))
    data['target'] = y_pred
    
    data = data.drop(['group_id', 'doc_id'], axis=1)

    data.to_csv("submission.csv", index=False)
    
    info = np.unique(data['target'], return_counts=True)
    
    if info[0].shape[0] > 1:
        
        print('0: {}, 1: {}'.format(info[1][0], info[1][1]))
        if info[1][1] > 6000 or info[1][1] < 2500:
            print('Your submisson is shit')
#         elif info[1][1] > 4500:
#             print('Your submisson is probably shit')
    else:
        print('There are only {} in submission'.format(info[0][0]))
        
    return data

In [17]:
def predict(X_train, X_test, train_target, model, scaler=None, th=0.30, **kwargs):
    
    curr_model = model(**kwargs)
    
    if scaler is not None:
        
        your_scaler = scaler()
        your_scaler.fit(X_train)
        X_train = your_scaler.transform(X_train)
        X_test = your_scaler.transform(X_test)
        
    curr_model.fit(X_train, train_target)
#     y_pred = curr_model.predict(X_test)
#     th = 0.35
    print('Threshold = ', th )#,'Params = ', *kwargs.values)
    y_pred = [0 if val < th else 1 for val in curr_model.predict_proba(X_test)[:,1]]
    return y_pred

In [18]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/misha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [19]:
morph = pymorphy2.MorphAnalyzer()
lemmatizer = WordNetLemmatizer()

In [20]:
# Лемматизация 

def str_parser(words_): 
    global j 
    new_string = '' 

    for i in re.findall(r'\b[а-я]{1,20}\b', words_): 
        new_string += (morph.parse(i)[0].normal_form) + ' ' 

    for i in re.findall(r'\b[a-z]{1,20}\b', words_): 
        new_string += (lemmatizer.lemmatize(i)) + ' ' 

    j += 1 
    if(j % 1000 == 0): 
        print(j, '/28317 loaded') 

    return new_string

In [21]:
def lemmatization(df, col_name='title'):
    
    return df[col_name].apply(lambda x: str_parser(x))

In [22]:
def stemming_titles(df, col_name='title'):
    stemming = Porter
    return df[col_name].apply(lambda x: stemming.stem(x))

In [23]:
def get_train_test_():

    df_train = pd.read_csv('data/core_train.csv')
    df_test = pd.read_csv('data/core_test.csv')
    df_train.fillna(' ', inplace=True)
    df_test.fillna(' ', inplace=True)
    titles_df = pd.concat([df_train, df_test],ignore_index = True)
    titles_df.drop(columns = {'Unnamed: 0'}, inplace = True)

    return titles_df

In [24]:
# Рассчитывает расстояния между документами берет 25 наименьших и сохраняет numpy ndarray в файл

def features_save_new(vec_type=1, way='tfidf', **kwargs):
    
    df = pd.read_csv('./data/unversal_table.csv')
    df.fillna(' ', inplace=True)
    corpus = df['title'].values
#     corpus = df[df.group_id == group_num]['title'].values
#     print(corpus)
    if way == 'countw':
        
        if vec_type == 1:
            vectorizer = CountVectorizer(max_features=max_f)
            X = vectorizer.fit_transform(corpus)

        elif vec_type ==2:
            vectorizer2 = CountVectorizer(max_features=max_f)
            X = vectorizer2.fit_transform(corpus)
            
    elif way == 'tfidf': 
        
#         vectorizer = TfidfVectorizer(**kwargs)
#         X = vectorizer.fit_transform(corpus)
        X = get_bm25_weights(corpus, n_jobs=-1)
    X = X.toarray()
    
    length = 0
    
    for group_num in np.unique(df['group_id']):
        
        group_length = len(df[df.group_id==group_num])
        group_titles = X[length:length + group_length]
        length += group_length
        
        features = count_distances(group_titles)

        np.save('group_features/{}'.format(group_num), features)
        if group_num % 50 == 0:
            print('Скачалась группа:', group_num)
        
    return features

In [56]:
# Рассчитывает расстояния между документами берет 25 наименьших и сохраняет numpy ndarray в файл

def features_save_new(vec_type=1, way='tfidf', **kwargs):
    
    df = pd.read_csv('./data/unversal_table.csv')
    df.fillna(' ', inplace=True)
    
    corpus = df['title'].values
    lens = df['len'].values
#     s = df['_num'].values
#     hrefs = df['href_num'].values
    
#     corpus = df[df.group_id == group_num]['title'].values
#     print(corpus)
    if way == 'countw':
        
        if vec_type == 1:
            vectorizer = CountVectorizer(max_features=max_f)
            X = vectorizer.fit_transform(corpus)

        elif vec_type ==2:
            vectorizer2 = CountVectorizer(max_features=max_f)
            X = vectorizer2.fit_transform(corpus)
            
    elif way == 'tfidf': 
        
        vectorizer = TfidfVectorizer(**kwargs)
        X = vectorizer.fit_transform(corpus)
#         X = get_bm25_weights(corpus, n_jobs=-1)
    X = X.toarray()
    print(X.shape, lens.shape)
    length = 0
    
    for group_num in np.unique(df['group_id']):
        
        group_length = len(df[df.group_id==group_num])
        
        group_titles = X[length:length + group_length]
        body_len = lens[length:length + group_length]
#         s_num = s[length:length + group_length]
#         hrefs_num = hrefs[length:length + group_length]
        
        length += group_length
        
        features = count_distances(group_titles)

        features = np.hstack((features, body_len[:, np.newaxis]))
        
        np.save('group_features/{}'.format(group_num), features)
        if group_num % 50 == 0:
            print('Скачалась группа:', group_num)
        
    return features

In [26]:
dbscan = DBSCAN(eps = 0.65, metric = 'cosine')
clf = IsolationForest()

In [27]:
# Рассчет расстояний по матрице встречаемости (косинусная метрика)

def count_distances(docs):
#     print(docs)cosine
# correlation
    distance = cdist(docs, docs, 'correlation')
#     distance = cdist(docs, docs, 'jaccard')
#     distance = cdist(docs, docs, 'cosine')
#     print(distance,'----------')
    res = np.asarray([np.concatenate((vec[:num],vec[num+1:])) for num, vec in enumerate(distance)])
    
    stats = np.hstack((np.mean(res, axis=1)[:, np.newaxis], np.std(res, axis=1)[:, np.newaxis], \
                       np.median(res, axis=1)[:, np.newaxis]))
    
    res = np.sort(res)[:, :25]
#     stats = np.hstack((np.mean(res, axis=1)[:, np.newaxis], np.std(res, axis=1)[:, np.newaxis], \
#                        np.median(res, axis=1)[:, np.newaxis]))
    res = np.hstack((res, stats))
    
#     res = np.flip(res, axis=1)[:, :25]
#     info = 15 - res.shape[1]
#     if info > 0:
#         z = np.zeros(shape=(res.shape[0], info))
#         res = np.hstack((z, res))
    
    return res

In [28]:
# def count_distances(docs):
# # DBSCAN ADDED new_feature: DBSCAN -> -1 ==> 0
# #                           DBSCAN ->  1 ==> 1
#     f = dbscan.fit_predict(docs)
    
#     distance = cdist(docs, docs, 'correlation')
#     res = np.asarray([np.concatenate((vec[:num],vec[num+1:])) for num, vec in enumerate(distance)])
#     res = np.nan_to_num(res, nan=0, posinf=1, neginf=0)
    
    
#     stats = np.hstack((np.mean(res,axis = 1)[:,np.newaxis], np.std(res, axis=1)[:,np.newaxis], \
#                      np.median(res,axis = 1)[:, np.newaxis]))
    
#     f2 = clf.fit_predict(res)
        
#     for i in range(len(f)):
        
#         f[i] = i

#     for i in range(len(f2)):
        
#         f2[i] = i

#     res = np.sort(res)[:, :25]
#     res = np.hstack((res, stats))
#     res = np.hstack((f[:, np.newaxis], res))
#     res = np.hstack((f2[:, np.newaxis], res))
    
#     return res

In [57]:
# Проходит по заданным группам и сохраняет признаки(расстояния)

def main_parser_and_saver(**kwargs):
    
#     for group_num in range(start_group, finish_group + 1):

#         features_save(group_num, vec_type=vec_type)
#         print('Скачалась группа:', group_num)
    features_save_new(**kwargs)
    return True

In [30]:
# Создает X_train, X_test, train_target

def prepare_data(**kwargs):
    
    X_train = all_group_feature_list(1, 129)
    X_test = all_group_feature_list(130, 309)
    
    d = pd.read_csv('./data/train_groups.csv')
    train_target = d['target']
    
    X_train = np.nan_to_num(X_train, nan=0, posinf=1, neginf=0)
    X_test = np.nan_to_num(X_test, nan=0, posinf=1, neginf=0)
    
    return X_train, train_target, X_test

In [31]:
# Загружает файлы с признаками документов по группам

def all_group_feature_list(start_group, finish_group):
    
    res = np.load('group_features/{}.npy'.format(start_group))
  
    for group_num in range(start_group + 1, finish_group + 1):
#         res += ndarray_to_list(np.load('group_features/{}.npy'.format(group_num)))
        
        t = np.load('group_features/{}.npy'.format(group_num))
#         print(res.shape, t.shape, group_num)
        res = np.vstack((res, t))
        
    return res

In [59]:
X_train_good, _ , X_test_good =  prepare_data()

In [ ]:
X_train_good.shape

In [81]:
X_train_53, X_test_53 = X_train, X_test

In [258]:
X_train = np.hstack((X_train_53, np.mean(X_train_old, axis=1)[:, np.newaxis],\
                   np.std(X_train_old, axis=1)[:, np.newaxis], np.median(X_train_old, axis=1)[:, np.newaxis]))
X_test = np.hstack((X_test_53, np.mean(X_test_old, axis=1)[:, np.newaxis],\
                   np.std(X_test_old, axis=1)[:, np.newaxis], np.median(X_test_old, axis=1)[:, np.newaxis]))

In [58]:
%%time
# main_parser_and_saver(1, 129)
# ngram_range=(1,2)
# min_df=2, 
main_parser_and_saver(stop_words='english')

(28317, 33091) (28317,)
Скачалась группа: 50
Скачалась группа: 100
Скачалась группа: 150
Скачалась группа: 200
Скачалась группа: 250
Скачалась группа: 300
CPU times: user 2min 47s, sys: 6.29 s, total: 2min 53s
Wall time: 3min 28s


True

In [300]:
X_train_body, y_train, X_test_body = prepare_data()

In [61]:
X_train_old, y_train_old, X_test_old, groups_train_old, groups_test_old = easy_launch()
groups_train = pd.read_csv('data/train_groups.csv')['group_id']

doc titles dict len = 28026
(11690, 25) (11690,) (11690,)
(16627, 25) (16627,)


In [60]:
X_train = np.hstack((X_train_good, X_train_old))
X_test = np.hstack((X_test_good, X_test_old))

NameError: name 'X_train_old' is not defined

In [405]:
X_train.shape

(11690, 54)

In [407]:
%%time
best = validation(X_train, y_train, GradientBoostingClassifier, params, GroupKFold, groups=groups_train, thresholds=th)

{'learning_rate': 0.06, 'n_estimators': 150}
th =  0.4
0.7445936854101338
--------max-------
(0.7445936854101338, {'learning_rate': 0.06, 'n_estimators': 150}, 0.4)
CPU times: user 54 s, sys: 68.8 ms, total: 54.1 s
Wall time: 54.1 s


In [408]:
y_pred1 = predict(X_train, X_test, y_train, GradientBoostingClassifier, StandardScaler, best[2], **best[1])
data = save_submission(y_pred1)

Threshold =  0.4
len data =  16627
0: 11729, 1: 4898


In [436]:
best = validation(X_train, y_train, CatBoostClassifier, params, GroupKFold, groups=groups_train, thresholds=th)

{'learning_rate': 0.06, 'n_estimators': 150}
th =  0.34
0:	learn: 0.6468683	total: 74.5ms	remaining: 11.1s
1:	learn: 0.6055122	total: 81.5ms	remaining: 6.03s
2:	learn: 0.5704531	total: 88.3ms	remaining: 4.33s
3:	learn: 0.5392777	total: 95.3ms	remaining: 3.48s
4:	learn: 0.5060802	total: 103ms	remaining: 2.99s
5:	learn: 0.4831252	total: 110ms	remaining: 2.64s
6:	learn: 0.4605093	total: 117ms	remaining: 2.39s
7:	learn: 0.4460273	total: 124ms	remaining: 2.19s
8:	learn: 0.4295115	total: 131ms	remaining: 2.06s
9:	learn: 0.4176897	total: 138ms	remaining: 1.94s
10:	learn: 0.4064333	total: 146ms	remaining: 1.84s
11:	learn: 0.3968633	total: 153ms	remaining: 1.76s
12:	learn: 0.3888477	total: 161ms	remaining: 1.69s
13:	learn: 0.3826969	total: 168ms	remaining: 1.63s
14:	learn: 0.3771506	total: 175ms	remaining: 1.57s
15:	learn: 0.3727197	total: 182ms	remaining: 1.52s
16:	learn: 0.3678261	total: 189ms	remaining: 1.48s
17:	learn: 0.3637114	total: 196ms	remaining: 1.44s
18:	learn: 0.3591429	total: 203m

22:	learn: 0.3349122	total: 219ms	remaining: 1.21s
23:	learn: 0.3322883	total: 231ms	remaining: 1.21s
24:	learn: 0.3301780	total: 240ms	remaining: 1.2s
25:	learn: 0.3276415	total: 250ms	remaining: 1.19s
26:	learn: 0.3258595	total: 258ms	remaining: 1.18s
27:	learn: 0.3243666	total: 269ms	remaining: 1.17s
28:	learn: 0.3221467	total: 279ms	remaining: 1.17s
29:	learn: 0.3206705	total: 288ms	remaining: 1.15s
30:	learn: 0.3195920	total: 297ms	remaining: 1.14s
31:	learn: 0.3179188	total: 305ms	remaining: 1.13s
32:	learn: 0.3170773	total: 313ms	remaining: 1.11s
33:	learn: 0.3155846	total: 322ms	remaining: 1.1s
34:	learn: 0.3141604	total: 331ms	remaining: 1.09s
35:	learn: 0.3132116	total: 340ms	remaining: 1.08s
36:	learn: 0.3123653	total: 350ms	remaining: 1.07s
37:	learn: 0.3115486	total: 360ms	remaining: 1.06s
38:	learn: 0.3105977	total: 369ms	remaining: 1.05s
39:	learn: 0.3098810	total: 378ms	remaining: 1.04s
40:	learn: 0.3091373	total: 386ms	remaining: 1.03s
41:	learn: 0.3081933	total: 394ms

33:	learn: 0.3065418	total: 292ms	remaining: 998ms
34:	learn: 0.3052314	total: 302ms	remaining: 994ms
35:	learn: 0.3044627	total: 310ms	remaining: 980ms
36:	learn: 0.3032693	total: 319ms	remaining: 973ms
37:	learn: 0.3023669	total: 326ms	remaining: 960ms
38:	learn: 0.3012199	total: 335ms	remaining: 955ms
39:	learn: 0.3001265	total: 343ms	remaining: 942ms
40:	learn: 0.2994686	total: 352ms	remaining: 935ms
41:	learn: 0.2987241	total: 358ms	remaining: 922ms
42:	learn: 0.2980930	total: 368ms	remaining: 915ms
43:	learn: 0.2975187	total: 375ms	remaining: 903ms
44:	learn: 0.2966415	total: 384ms	remaining: 896ms
45:	learn: 0.2956300	total: 391ms	remaining: 884ms
46:	learn: 0.2952195	total: 400ms	remaining: 878ms
47:	learn: 0.2946629	total: 408ms	remaining: 867ms
48:	learn: 0.2941075	total: 415ms	remaining: 856ms
49:	learn: 0.2938473	total: 422ms	remaining: 843ms
50:	learn: 0.2929700	total: 430ms	remaining: 835ms
51:	learn: 0.2923109	total: 437ms	remaining: 824ms
52:	learn: 0.2920328	total: 446

54:	learn: 0.3116398	total: 413ms	remaining: 714ms
55:	learn: 0.3110140	total: 422ms	remaining: 708ms
56:	learn: 0.3106428	total: 430ms	remaining: 701ms
57:	learn: 0.3099832	total: 437ms	remaining: 694ms
58:	learn: 0.3092605	total: 446ms	remaining: 689ms
59:	learn: 0.3084539	total: 454ms	remaining: 681ms
60:	learn: 0.3079113	total: 463ms	remaining: 675ms
61:	learn: 0.3074251	total: 470ms	remaining: 667ms
62:	learn: 0.3070431	total: 477ms	remaining: 659ms
63:	learn: 0.3067084	total: 484ms	remaining: 651ms
64:	learn: 0.3064222	total: 492ms	remaining: 643ms
65:	learn: 0.3063232	total: 499ms	remaining: 635ms
66:	learn: 0.3058885	total: 506ms	remaining: 626ms
67:	learn: 0.3056319	total: 513ms	remaining: 618ms
68:	learn: 0.3052525	total: 520ms	remaining: 610ms
69:	learn: 0.3049580	total: 529ms	remaining: 604ms
70:	learn: 0.3045303	total: 536ms	remaining: 596ms
71:	learn: 0.3038648	total: 544ms	remaining: 590ms
72:	learn: 0.3034800	total: 551ms	remaining: 581ms
73:	learn: 0.3031356	total: 560

84:	learn: 0.2798223	total: 608ms	remaining: 465ms
85:	learn: 0.2794531	total: 618ms	remaining: 460ms
86:	learn: 0.2791271	total: 625ms	remaining: 453ms
87:	learn: 0.2787077	total: 633ms	remaining: 446ms
88:	learn: 0.2784991	total: 641ms	remaining: 439ms
89:	learn: 0.2782682	total: 647ms	remaining: 432ms
90:	learn: 0.2779564	total: 654ms	remaining: 424ms
91:	learn: 0.2775302	total: 662ms	remaining: 417ms
92:	learn: 0.2771445	total: 669ms	remaining: 410ms
93:	learn: 0.2768750	total: 676ms	remaining: 403ms
94:	learn: 0.2765687	total: 683ms	remaining: 396ms
95:	learn: 0.2764049	total: 691ms	remaining: 388ms
96:	learn: 0.2762798	total: 697ms	remaining: 381ms
97:	learn: 0.2760536	total: 704ms	remaining: 373ms
98:	learn: 0.2757895	total: 711ms	remaining: 366ms
99:	learn: 0.2754875	total: 717ms	remaining: 359ms
100:	learn: 0.2751746	total: 724ms	remaining: 351ms
101:	learn: 0.2750561	total: 731ms	remaining: 344ms
102:	learn: 0.2746040	total: 738ms	remaining: 337ms
103:	learn: 0.2743646	total:

...

4:	learn: 0.5060802	total: 45.4ms	remaining: 1.32s
5:	learn: 0.4831252	total: 55.1ms	remaining: 1.32s
6:	learn: 0.4605093	total: 62.8ms	remaining: 1.28s
7:	learn: 0.4460273	total: 70.2ms	remaining: 1.25s
8:	learn: 0.4295115	total: 77.5ms	remaining: 1.21s
9:	learn: 0.4176897	total: 86.1ms	remaining: 1.21s
10:	learn: 0.4064333	total: 93.5ms	remaining: 1.18s
11:	learn: 0.3968633	total: 102ms	remaining: 1.17s
12:	learn: 0.3888477	total: 110ms	remaining: 1.16s
13:	learn: 0.3826969	total: 118ms	remaining: 1.14s
14:	learn: 0.3771506	total: 125ms	remaining: 1.12s
15:	learn: 0.3727197	total: 134ms	remaining: 1.12s
16:	learn: 0.3678261	total: 141ms	remaining: 1.1s
17:	learn: 0.3637114	total: 150ms	remaining: 1.1s
18:	learn: 0.3591429	total: 157ms	remaining: 1.08s
19:	learn: 0.3563613	total: 166ms	remaining: 1.08s
20:	learn: 0.3539734	total: 173ms	remaining: 1.06s
21:	learn: 0.3519335	total: 182ms	remaining: 1.06s
22:	learn: 0.3499149	total: 189ms	remaining: 1.04s
23:	learn: 0.3468423	total: 198m

25:	learn: 0.3276415	total: 213ms	remaining: 1.02s
26:	learn: 0.3258595	total: 221ms	remaining: 1.01s
27:	learn: 0.3243666	total: 230ms	remaining: 1s
28:	learn: 0.3221467	total: 238ms	remaining: 993ms
29:	learn: 0.3206705	total: 246ms	remaining: 982ms
30:	learn: 0.3195920	total: 255ms	remaining: 979ms
31:	learn: 0.3179188	total: 262ms	remaining: 967ms
32:	learn: 0.3170773	total: 271ms	remaining: 961ms
33:	learn: 0.3155846	total: 279ms	remaining: 952ms
34:	learn: 0.3141604	total: 288ms	remaining: 945ms
35:	learn: 0.3132116	total: 295ms	remaining: 935ms
36:	learn: 0.3123653	total: 304ms	remaining: 929ms
37:	learn: 0.3115486	total: 312ms	remaining: 919ms
38:	learn: 0.3105977	total: 321ms	remaining: 912ms
39:	learn: 0.3098810	total: 328ms	remaining: 902ms
40:	learn: 0.3091373	total: 336ms	remaining: 894ms
41:	learn: 0.3081933	total: 344ms	remaining: 885ms
42:	learn: 0.3077375	total: 353ms	remaining: 878ms
43:	learn: 0.3068101	total: 360ms	remaining: 868ms
44:	learn: 0.3062083	total: 368ms	

54:	learn: 0.2904242	total: 452ms	remaining: 781ms
55:	learn: 0.2899397	total: 462ms	remaining: 775ms
56:	learn: 0.2895620	total: 470ms	remaining: 767ms
57:	learn: 0.2889820	total: 480ms	remaining: 761ms
58:	learn: 0.2885963	total: 487ms	remaining: 751ms
59:	learn: 0.2881033	total: 496ms	remaining: 744ms
60:	learn: 0.2875505	total: 503ms	remaining: 734ms
61:	learn: 0.2869288	total: 513ms	remaining: 727ms
62:	learn: 0.2864249	total: 520ms	remaining: 718ms
63:	learn: 0.2861084	total: 530ms	remaining: 712ms
64:	learn: 0.2857665	total: 538ms	remaining: 703ms
65:	learn: 0.2854876	total: 548ms	remaining: 697ms
66:	learn: 0.2852173	total: 557ms	remaining: 690ms
67:	learn: 0.2847502	total: 566ms	remaining: 682ms
68:	learn: 0.2843764	total: 575ms	remaining: 674ms
69:	learn: 0.2839872	total: 583ms	remaining: 666ms
70:	learn: 0.2836474	total: 594ms	remaining: 661ms
71:	learn: 0.2830473	total: 604ms	remaining: 654ms
72:	learn: 0.2826230	total: 612ms	remaining: 646ms
73:	learn: 0.2822872	total: 621

73:	learn: 0.3031356	total: 705ms	remaining: 724ms
74:	learn: 0.3025530	total: 712ms	remaining: 712ms
75:	learn: 0.3022806	total: 722ms	remaining: 703ms
76:	learn: 0.3019481	total: 730ms	remaining: 692ms
77:	learn: 0.3015888	total: 740ms	remaining: 683ms
78:	learn: 0.3014325	total: 748ms	remaining: 672ms
79:	learn: 0.3008923	total: 756ms	remaining: 662ms
80:	learn: 0.3005745	total: 765ms	remaining: 651ms
81:	learn: 0.3001694	total: 773ms	remaining: 641ms
82:	learn: 0.2997926	total: 782ms	remaining: 631ms
83:	learn: 0.2992175	total: 791ms	remaining: 621ms
84:	learn: 0.2989152	total: 799ms	remaining: 611ms
85:	learn: 0.2984671	total: 807ms	remaining: 601ms
86:	learn: 0.2978964	total: 816ms	remaining: 591ms
87:	learn: 0.2973528	total: 824ms	remaining: 580ms
88:	learn: 0.2971252	total: 832ms	remaining: 570ms
89:	learn: 0.2969415	total: 839ms	remaining: 560ms
90:	learn: 0.2967295	total: 849ms	remaining: 550ms
91:	learn: 0.2966421	total: 856ms	remaining: 540ms
92:	learn: 0.2962774	total: 865

95:	learn: 0.2764049	total: 852ms	remaining: 479ms
96:	learn: 0.2762798	total: 863ms	remaining: 471ms
97:	learn: 0.2760536	total: 872ms	remaining: 462ms
98:	learn: 0.2757895	total: 880ms	remaining: 453ms
99:	learn: 0.2754875	total: 888ms	remaining: 444ms
100:	learn: 0.2751746	total: 896ms	remaining: 435ms
101:	learn: 0.2750561	total: 903ms	remaining: 425ms
102:	learn: 0.2746040	total: 912ms	remaining: 416ms
103:	learn: 0.2743646	total: 919ms	remaining: 406ms
104:	learn: 0.2741955	total: 928ms	remaining: 398ms
105:	learn: 0.2739077	total: 936ms	remaining: 389ms
106:	learn: 0.2736346	total: 945ms	remaining: 380ms
107:	learn: 0.2731683	total: 953ms	remaining: 371ms
108:	learn: 0.2727446	total: 962ms	remaining: 362ms
109:	learn: 0.2724391	total: 971ms	remaining: 353ms
110:	learn: 0.2722336	total: 979ms	remaining: 344ms
111:	learn: 0.2719140	total: 988ms	remaining: 335ms
112:	learn: 0.2716138	total: 996ms	remaining: 326ms
113:	learn: 0.2713280	total: 1s	remaining: 317ms
114:	learn: 0.270969

In [297]:
params = {'learning_rate': [0.06], 
'n_estimators': [73]} 
th = [0.34]

Threshold =  0.3
len data =  16627
0: 10767, 1: 5860


In [287]:
params = {'learning_rate': [0.06], 
#           'n_estimators': [120, 130, 140, 150]} 
#           'n_estimators': [50, 55, 60, 65, 70, 75,80,85, 90, 95, 100, 105, 110, 115, 120]}
          'n_estimators': [100, 150, 200, 250, 300]} 
th = [0.39 , 0.4] 

In [239]:
params = {'learning_rate': [0.05],
#           'n_estimators': [372, 374]}
          'n_estimators': [250]}
th = [0.3]

In [435]:
params = {'learning_rate': [0.06],
          'n_estimators' : [150]}
# th = [0.4]
# th = [0.26, 0.27, 0.28, 0.29, 0.3, 0.31,0.32,0.33]  
th = [0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42]

In [243]:
params = {'learning_rate': [0.06],
#           'n_estimators': [372, 374]}
          'n_estimators': [80]}
th = [0.38]

In [429]:
params = {'algorithm': ['ball_tree', 'brute', 'kd_tree'],
# params = {'algorithm': ['ball_tree'],
# params = {'algorithm': ['auto'],
#           'leaf_size': [5, 10, 20, 40],
#           'n_neighbors': [30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80],
          'n_neighbors': [10, 15, 20],
#           'weights': [smart_weights],
          'p': [1],
          'n_jobs': [-1]}
th = [0.35] 
# th = [0.26, 0.27, 0.28, 0.29, 0.3, 0.31,0.32,0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42]

In [433]:
# params = {'algorithm': ['brute'],
params = {'algorithm': ['ball_tree'],
# params = {'algorithm': ['auto'],
#           'leaf_size': [10],
          'n_neighbors': [75],
#           'weights': [smart_weights],
          'p': [1],
          'n_jobs': [-1]}
th = [0.27, 0.28, 0.29, 0.3,0.31,0.32,0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42]

In [421]:
th = [0.41] 
params = {'n_estimators': [100, 200, 300, 400, 800], 
#           'criterion': ['gini', 'entropy'], 
          'criterion': ['gini'], 
          'max_depth': [9], 
#           'max_depth': [10, 15, None], 
          'n_jobs': [-1]}

In [443]:
params = {'n_estimators': [100], 
          'criterion': ['entropy'], 
          'max_depth': [10], 
          'n_jobs': [-1]}
# th=[0.41]
th = [0.3,0.31,0.32,0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45]

In [445]:
params = {'n_estimators': [800], 
          'criterion': ['gini'], 
#           'criterion': ['gini'], 
          'max_depth': [9], 
#           'max_depth': [10, 15, None], 
          'n_jobs': [-1]}
th=[0.36]
# th = [0.26, 0.27, 0.28, 0.29, 0.3, 0.29, 0.3,0.31,0.32,0.33, 0.34, 0.35, 0.36, 0.37, 0.38]# 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45]

In [446]:
# for i in range(5):
best = validation(main_res_train, y_train, RandomForestClassifier, params, GroupKFold, groups=groups_train, thresholds=th)

{'n_estimators': 800, 'criterion': 'gini', 'max_depth': 9, 'n_jobs': -1}
th =  0.36
0.7415311086141798
--------max-------
(0.7415311086141798, {'n_estimators': 800, 'criterion': 'gini', 'max_depth': 9, 'n_jobs': -1}, 0.36)


In [73]:
y_pred3 = predict(X_train, X_test, y_train, RandomForestClassifier, StandardScaler, best[2], **best[1],)
data = save_submission(y_pred3)

Threshold =  0.39
len data =  16627
0: 11699, 1: 4928


In [434]:
best = validation(X_train, y_train, KNeighborsClassifier, params, GroupKFold, groups=groups_train, thresholds=th)

{'algorithm': 'ball_tree', 'n_neighbors': 75, 'p': 1, 'n_jobs': -1}
th =  0.27
0.7140179087677511
th =  0.28
0.7140179087677511
th =  0.29
0.7151617727398918
th =  0.3
0.7168017085958605
th =  0.31
0.7158697322613526
th =  0.32
0.7158697322613526
th =  0.33
0.7178909249189525
th =  0.34
0.7187334838034716
th =  0.35
0.7217532555870074
th =  0.36
0.7217532555870074
th =  0.37
0.7201643427182952
th =  0.38
0.7198592669838952
th =  0.39
0.7191443931375207
th =  0.4
0.7191443931375207
th =  0.41
0.7145801957876023
th =  0.42
0.7156183745610593
--------max-------
(0.7217532555870074, {'algorithm': 'ball_tree', 'n_neighbors': 75, 'p': 1, 'n_jobs': -1}, 0.35)


In [494]:
best = validation(main_res_train, y_train, SGDClassifier, params, GroupKFold, folds_num=15, groups=groups_train, thresholds=th)

{'loss': 'log', 'alpha': 0.01}
th =  0.26
0.7496873281062211
{'loss': 'log', 'alpha': 0.005}
th =  0.26
0.7511826264581546
{'loss': 'log', 'alpha': 0.001}
th =  0.26
0.7529907621753301
{'loss': 'log', 'alpha': 0.0005}
th =  0.26
0.7521764349044318
{'loss': 'log', 'alpha': 0.0001}
th =  0.26
0.7556957621124691
--------max-------
(0.7556957621124691, {'loss': 'log', 'alpha': 0.0001}, 0.26)


In [495]:
y_pred3 = predict(main_res_train, main_res_test, y_train, SGDClassifier, StandardScaler, best[2], **best[1],)
data = save_submission(y_pred3)

Threshold =  0.26
len data =  16627
0: 11479, 1: 5148


In [456]:
params = {'loss': ['log'],
          'alpha': [0.0001]}
#           'alpha': [0.01, 0.005, 0.001, 0.0005, 0.0001]}
th = [0.26]
# th = [0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3,0.31,0.32,0.33, 0.34]

In [249]:
word, num = np.unique(body[10024].split(), return_counts=True)

In [244]:
lambda b: list(zip(word, num))

<function __main__.<lambda>(x)>

In [253]:
j=0

In [395]:
def body_change(text):
    global j
    word, num = np.unique(text.split(), return_counts=True)
    words = sorted(list(zip(word, num)), key=lambda pair: pair[1], reverse=True)[:40]
    text = ' '.join(list(map(lambda x: x[0], words)))
    j+=1 
    print(j)
    return text

In [69]:
# titles_df = pd.DataFrame()
# titles_df['body'] = main_df['body'].apply(lambda x: body_change(x))

In [398]:
titles = pd.DataFrame({'doc_id': main_df['doc_id'] ,
                       'title': titles_df['body'],
#                        'title': titles_df['title'] + titles_df['h1'],
#                        'title': titles_df['h2'] + titles_df['h3'] + titles_df['a'],
                       'group_id': main_df['group_id']})
titles.to_csv('./data/unversal_table.csv', index=False)

In [35]:
%%time
titles_df = pd.read_csv('./data/unversal_table.csv')
print('OK1')
main_df = pd.read_csv('data/CORE.csv')
print('OK2')
main_df['body'] = titles_df['title']
print('OK3')
main_df.to_csv('data/CORE.csv')
print('OK4')

OK1
OK2
OK3
OK4
CPU times: user 2min 1s, sys: 35.9 s, total: 2min 37s
Wall time: 3min 33s


In [115]:
titles_df = main_df[['doc_id', 'title', 'group_id', 'h1', 'strong']]

In [ ]:
titles_df['strong'] = titles_df['strong'].apply(lambda x: ' '.join(np.unique(x.lower().split())))

In [109]:
titles_df.to_csv('./data/unversal_table.csv', index=False)

In [51]:
df_train = pd.read_csv('data/core_train.csv')
df_test = pd.read_csv('data/core_test.csv')

df_train.fillna(' ', inplace=True)
df_test.fillna(' ', inplace=True)

df_train_strong = pd.read_csv('data/TRAIN_NUMBERS_TITLES.csv')
df_test_strong = pd.read_csv('data/TEST_NUMBERS_TITLES.csv')

df_train_strong.fillna(' ', inplace=True)
df_test_strong.fillna(' ', inplace=True)

In [53]:
titles_all = pd.concat([df_train, df_test])
titles_strong = pd.concat([df_train_strong, df_test_strong])

In [90]:
titles_df = pd.merge(titles_all, titles_strong[['doc_id', 'strong']], on='doc_id', how='inner')

In [87]:
titles_df['strong'] = titles_df['strong'].apply(lambda x: ' '.join(np.unique(x.lower().split())))

In [370]:
# titles_h2h3a = pd.read_csv('./data/no_bad_words_table.csv')

In [383]:
# titles_df = pd.merge(titles_all, titles_h2h3a[['doc_id', 'title']], on='doc_id', how='inner')

In [385]:
titles_df2 = pd.DataFrame({'doc_id': titles_df['doc_id'] ,
                       'title': titles_df['title_x'] + titles_df['h1'],
#                        'title': titles_df['title'] + titles_df['h1'],
#                        'title': titles_df['h2'] + titles_df['h3'] + titles_df['a'],
                       'group_id': titles_df['group_id']})

titles_df2['title'] = stemming_titles(titles_df2)

In [70]:
titles = pd.DataFrame({'doc_id': titles_df['doc_id'] ,
                       'title': titles_df['title'],
#                        'title': titles_df['title'] + titles_df['h1'],
#                        'title': titles_df['h2'] + titles_df['h3'] + titles_df['a'],
                       'group_id': titles_df['group_id']})

titles.fillna(' ', inplace=True)
j = 0

# titles['title'] = stemming_titles(titles)
# titles['title'] = lemmatization(titles)

# titles['title']

NameError: name 'titles_df' is not defined

In [100]:
titles['title'][0]

'ваза зам подшипник ступица нива '

In [101]:
titles.to_csv('./data/unversal_table.csv', index=False)

In [298]:
titles.fillna(' ', inplace=True)

In [52]:
corpus = [
     'This is the first document.',
     'This document is the second document.',
     'And this is the third one.',
     'Is this the first document?',
     ]

# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(corpus)
# print(vectorizer.get_feature_names())
# ['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [53]:
corpus

['This is the first document.',
 'This document is the second document.',
 'And this is the third one.',
 'Is this the first document?']

In [54]:
iterat = iter_bm25_bow(corpus, n_jobs=-1)

In [183]:
titles_df= pd.read_csv('data/no_bad_words_table.csv')

In [38]:
type(X)

list

In [169]:
df['title_unique'][5]

'аккумулятор багажник балк бензонасос блок ваз вентилятор виде где генератор главн грант грм давлен дальш датчик двигател для жидкост задн зажиган зам запчаст инжектор как калин клапан колодок контакт коробк крышк лад ламп ларгус магазин масл машин мост опор охлажда охлажден панел передн печк подшипник предохранител прибор приор проверк пружин радиатор разва регулировк рел ремн ремонт рук рулев сайлентблок сайт салон сальник свеч сво систем снят спидометр стартер стеклоподъемник ступиц ступичн схем сцеплен топливн тормоз тормозн тюнинг фильтр цилиндр чита '

In [184]:
titles = pd.DataFrame({'doc_id': titles_df['doc_id'] ,
                       'title': titles_df['title_unique'],
#                        'title': titles_df['title'] + titles_df['h1'],
#                        'title': titles_df['h2'] + titles_df['h3'] + titles_df['a'],
                       'group_id': titles_df['group_id']})

In [180]:
new_df['title_unique'].name = 'title'

In [189]:
titles.fillna(' ', inplace=True)

In [154]:
# Стекинг !!!
def stacking(X_train, X_test, train_target, models, params, folds_gen_func, groups_num=10, thresholds=[0.27], **kwargs):
    
    scaler = StandardScaler()
        
    main_res_train = np.zeros(shape=(train_target.shape[0], 1))
    main_res_test = np.zeros(shape=(X_test.shape[0], 1))
    
    for model in models:
        
        print(params[model])        
        exact_model = model(**params[model]) 
        
        fold_generator = folds_gen_func(groups_num)
        
        res_train = np.zeros(train_target.shape[0])
        res_test = np.zeros(X_test.shape[0])
        
        for train_index, test_index in fold_generator.split(X_train, train_target, **kwargs):
            
            scaler.fit(X_train)
            print(scaler.transform(X_train[train_index]).shape, train_target[train_index].shape)
            exact_model.fit(scaler.transform(X_train[train_index]), train_target[train_index])

            y_pred_train = exact_model.predict_proba(scaler.transform(X_train[test_index]))
            res_train[test_index] = y_pred_train[:,1] 
            
            scaler.fit(X_test)
            y_pred_test = exact_model.predict_proba(scaler.transform(X_test))

            res_test += y_pred_test[:,1]
            
        res_test /= groups_num
        main_res_train = np.hstack((main_res_train, res_train[:, np.newaxis]))
        main_res_test = np.hstack((main_res_test, res_test[:, np.newaxis]))   

    return main_res_train, main_res_test

In [438]:
models = [GradientBoostingClassifier, RandomForestClassifier, KNeighborsClassifier]

In [439]:
params = {}
params[GradientBoostingClassifier] = {'learning_rate': 0.06,
                                      'n_estimators' : 150}
params[RandomForestClassifier] = {'n_estimators': 800, 
                                  'criterion': 'gini', 
                                  'max_depth': 9, 
                                  'n_jobs': -1}
params[KNeighborsClassifier] = {'algorithm': 'ball_tree',
                                'leaf_size': 10,
                                'n_neighbors': 80,
                                'p': 1,
                                'n_jobs': -1}
# params[SGDClassifier] = {'loss': 'log',
#                          'alpha': 0.005}

In [440]:
main_res_train, main_res_test = stacking(X_train, X_test, y_train, models, params, GroupKFold, groups=groups_train)

{'learning_rate': 0.06, 'n_estimators': 150}
(10523, 54) (10523,)
(10513, 54) (10513,)
(10513, 54) (10513,)
(10513, 54) (10513,)
(10524, 54) (10524,)
(10514, 54) (10514,)
(10526, 54) (10526,)
(10516, 54) (10516,)
(10549, 54) (10549,)
(10519, 54) (10519,)
{'n_estimators': 800, 'criterion': 'gini', 'max_depth': 9, 'n_jobs': -1}
(10523, 54) (10523,)
(10513, 54) (10513,)
(10513, 54) (10513,)
(10513, 54) (10513,)
(10524, 54) (10524,)
(10514, 54) (10514,)
(10526, 54) (10526,)
(10516, 54) (10516,)
(10549, 54) (10549,)
(10519, 54) (10519,)
{'algorithm': 'ball_tree', 'leaf_size': 10, 'n_neighbors': 80, 'p': 1, 'n_jobs': -1}
(10523, 54) (10523,)
(10513, 54) (10513,)
(10513, 54) (10513,)
(10513, 54) (10513,)
(10524, 54) (10524,)
(10514, 54) (10514,)
(10526, 54) (10526,)
(10516, 54) (10516,)
(10549, 54) (10549,)
(10519, 54) (10519,)


In [67]:
steming = Porter
morph = pymorphy2.MorphAnalyzer()
lemmatizer = WordNetLemmatizer()
#___________________________________________
def meta_tag(doc_id):
    
    result = str()
    file_ = open('./content/' + str(doc_id) + '.dat', 'r', encoding = 'utf-8')
    text_ = file_.read()
    soup = BeautifulSoup(text_, 'html')
    
    for i in soup.find_all('meta', attrs = {'name' : 'description'}):
        if i.find_all('content'):
            result += i.attrs['content'] + ' '

    result = result.lower()
    new_string = str()
    
    result = steming.stem(result)
    
    for i in re.findall(r'\b[а-я]{1,20}\b', result): 
        new_string += (morph.parse(i)[0].normal_form) + ' ' 

    for i in re.findall(r'\b[a-z]{1,20}\b', result):
        new_string += (lemmatizer.lemmatize(i)) + ' ' 
    
    
    return new_string

In [63]:
df = pd.read_csv('./data/unversal_table.csv')